# ACDC 2019-2020 : l'Autre Carré D'à Côté

alias "la zone à JPD"

alias "le plateau de Cournols-Olloix"

(pour faire du DS évidemment)

In [ ]:
import sys
import os
import math
import re
import datetime as dt
import pandas as pd
import numpy as np

from lxml import etree
import pyproj

import folium

import simplekml as skml # Simple KML generation API
import kmlcircle # Circle generation for KML generation

from shapely import geometry

from IPython.display import HTML

# Communs

In [ ]:
KInfValues = [np.inf, -np.inf]

In [ ]:
kmlNameSpaces = \
  { 'gx' : 'http://www.google.com/kml/ext/2.2',
    'kml' : 'http://www.opengis.net/kml/2.2',
    'atom' : 'http://www.w3.org/2005/Atom' }

In [ ]:
# Projection coordonnées sphériques (degrés)=> coordonnées planes (système cible au choix)
KProjWgs84 = pyproj.Proj(init='epsg:4326') # WGS 84 : long, lat en degrés

KProjUtm31  = pyproj.Proj(init='epsg:32631') # WGS 84 - UTM 31N : long, lat en m
KProjCc46   = pyproj.Proj(init='epsg:3946')  # RGF 93 - CC46    : long, lat en m
KProjLamb93 = pyproj.Proj(init='epsg:2154')  # RGF 93 - Lambert : long, lat en m

def geoProjeter(sCoords, srcProj, tgtProj): # sCoords : [0]=x=long, [1]=y=lat
    return pd.Series(pyproj.transform(srcProj, tgtProj, sCoords[0], sCoords[1]))

# Attribution des points aux observateurs 

Grille nettoyée / corrigée (déplacé points hors broussailles, forêt, bocage trop serré, ou trop loin accès ... quand possible ;
 supprimé points en forêt ou innaccessibles cause relief)

In [ ]:
# Version finale.
kmlRoot = etree.ElementTree().parse('ACDC/DS ACDC COURNOLS OLLOIX JPM v4.kml')

## 1) Chargement du polygône définissant la zone couverte

In [ ]:
# Zone à couvrir : On suppose que c'est le 1er polygône
plMark = kmlRoot.find('kml:Document/kml:Placemark', namespaces=kmlNameSpaces)
zonePoly = plMark.find('kml:Polygon/kml:outerBoundaryIs/kml:LinearRing/kml:coordinates',
                       namespaces=kmlNameSpaces).text.strip()
dfZonePoly = pd.DataFrame(data=[[float(v) for v in point.split(',')] for point in zonePoly.split(' ')],
                          columns=['long', 'lat', 'alt'])
dfZonePoly.head()

## 2) Chargement des points conservés

In [ ]:
def iterPlacemarks(kmlDoc, kmlNameSpaces):
    for pm in kmlDoc.findall('kml:Document/kml:Document/kml:Placemark', namespaces=kmlNameSpaces):
        name = pm.find('kml:name', namespaces=kmlNameSpaces).text
        desc = pm.find('kml:description', namespaces=kmlNameSpaces).text
        #print(name)
        long, lat, alt = pm.find('kml:Point/kml:coordinates', namespaces=kmlNameSpaces).text.split(',')
        yield dict(numero=int(name), description=desc, longitude=float(long), latitude=float(lat), altitude=float(alt))

In [ ]:
dfPoints = pd.DataFrame(data=list(iterPlacemarks(kmlRoot, kmlNameSpaces)))

In [ ]:
# 3) Ajout coordonnées UTM31
dfPoints[['longitude_utm', 'latitude_utm']] = \
  dfPoints[['longitude', 'latitude']].apply(geoProjeter, srcProj=KProjWgs84, tgtProj=KProjUtm31, axis='columns')
dfPoints[['longitude_utm', 'latitude_utm']] = \
  dfPoints[['longitude_utm', 'latitude_utm']].replace(KInfValues, np.nan) / 1000

In [ ]:
dfPoints.set_index('numero', inplace=True)
dfPoints['papier'] = None
dfPoints['naturalist'] = None
dfPoints = dfPoints.reindex(columns=['papier', 'naturalist', 'latitude', 'longitude', 'latitude_utm', 'longitude_utm',
                                     'altitude', 'description'])
len(dfPoints)

In [ ]:
dfPoints.head()

## 4) Attribution aux observateurs

### Contactés, mais pas volontaires

#### Liste François, Auvergne ...

* Christian Fargeix 7/3
* Nicole et Christian Taillandier 10/3
* ('Bruno Gilbert', 'lubrubelle@gmail.com', 'non'), # 11/03
* ('Marc Pommarel', 'pommarelmarc@neuf.fr', 'non'), # 11/03
* ('Gérard Lecoz', 'lecozgerard0818@orange.fr', 'non'), # 11/03
* ('Sabine Boursange' , 'sabine.boursange@lpo.fr', 'non') # 13/03
* ('Jean-Jacques Lallemant', 'jj.lallemant@gmail.com', 'non'), # 13/03
* ('Adèle Debaudoin', 'adele.debaudouin@outlook.fr', 'non', 'non', 'non'), # 14/03
* ('Olivier Gimel', 'gimelolivier@orange.fr', 'ok', 'non'), # Oui, puis non le 28/03 !

#### Liste glanée sur FA (données sérieuses sur les 3 communes ACDC)

* Luc Souret, luc83@orange.fr, 72 Chemin de Quinson - 83560 St Julien ; préretraire fin 2019, intéressé pour 2020 !
* Paul Nicolas, pbg.nicolas@gmail.com ; pas confiance dans la méthode
* Matthieu Bernard, matthieubernard8944@neuf.fr ; déménage prochainement à Culhat, pas le temps et trop loin


In [ ]:
# Infos oservateurs : Prénom + Nom, e-mail, statut de volontaire confirmé ?, présence confirmée le 31/03 ?
obseurs = \
[('François Guélin', 'francois.guelin@orange.fr', 'ok', 'oui'),
 ('Jean-Philippe Meuret', 'jpmeuret@free.fr', 'ok', 'oui'),
 ('Sylvain Sainnier', 'sainnier@gmail.com', 'ok', 'oui'), # formation 31/3 et 7 OK
 ('Gilles Saulas', 'gilles.saulas@orange.fr', 'ok', 'non'),
 ('Cyrille Jallageas', 'cyrisle@yahoo.fr', 'ok', 'oui'),
 ('Jean-François Carrias', 'jean-francois.carrias@orange.fr', 'ok', 'non'),
 ('Alex Clamens', 'clamens.alex@wanadoo.fr', 'ok', 'non'),
 ('Anne Citron', 'acitron@orange.fr', 'ok', 'non'),
 ('David Houston', 'd-houston@wanadoo.fr', 'ok', 'non'),
 ('Sandra Robert', 'sandlise@orange.fr', 'ok', 'oui',),
 ('Camille Fasolin', 'c.fasolin@gmail.com', 'ok', 'non'),
 ('Clément Rollant', 'clement.rollant@lpo.fr', 'ok', 'non'),
 ('Matthieu Clément', 'matthieu.clement@lpo.fr', 'ok', 'oui'),
 ('Laurent Maly', 'altla@orange.fr', 'ok', 'non'),
 ('Thibaut Brugerolle', 'tbrugerolle@hotmail.com', 'ok', 'non'),
 ('Pierre Tourret', 'pierre.tourret@wanadoo.fr', 'ok', 'oui'),
 ('Patrick Mougel', 'mougel.patrick@wanadoo.fr', 'ok', 'oui'),
 ('Cyril Brunel', 'cyrilb63@hotmail.fr', 'ok', 'oui'),
 ('Thyphaine Lyon', 'typhainelyon@gmail.com', 'ok', 'non'),
 ('Romain Riols', 'romain.riols@lpo.fr', 'ok', 'non')]

dfObseurs = pd.DataFrame(columns=['nom', 'eMail', 'statut', '31 mars'],
                         index=range(1, len(obseurs)+1), data=obseurs)

dfObseurs

In [ ]:
print('Observateurs volontaires :', len(dfObseurs))
print('* Les noms :', ', '.join(dfObseurs.nom))
print('* Les emails :', ', '.join(dfObseurs.eMail))

In [ ]:
df = dfObseurs[dfObseurs['31 mars'] != 'non']
print('Observateurs présents le 31 :', len(df))
print('* Les noms :', ', '.join(df.nom))
print('* Les emails :', ', '.join(df.eMail))

In [ ]:
# Réinitialisation de toutes les assignations
dfPoints['papier'] = None
dfPoints['naturalist'] = None

In [ ]:
# numPoints : liste des numéros de points
# papier : nom de l'observateur qui inventoriera ces points en mode "DS papier" (défaut : None=personne)
# naturalist  : idem en mode "DS Naturalist" (défaut : None=personne)
def assignerPoints(numPoints, papier=None, naturalist=None):
    
    assert papier is None or papier in list(dfObseurs.nom)
    assert naturalist is None or naturalist in list(dfObseurs.nom)
    if papier:
        assert dfPoints.loc[numPoints, 'papier'].isnull().all()
        dfPoints.loc[numPoints, 'papier'] = papier
    if naturalist:
        assert dfPoints.loc[numPoints, 'naturalist'].isnull().all()
        dfPoints.loc[numPoints, 'naturalist'] = naturalist

In [ ]:
list(range(195, 201))

In [ ]:
assignerPoints(papier='François Guélin', # OK, papier 02/03
               numPoints=[145, 146, 147, 160, 161, 162, 163, 164, 165, 177, 178, 179, # 02/03
                          195, 196, 197, 198, 199, 200, 109, 110, 126, 127, 128, 143, 144])  # 03/04

In [ ]:
assignerPoints(naturalist='Jean-Philippe Meuret', # 
               numPoints=[143, 144, 126, 127, 128, 109, 110, 112]) # OK, naturalist 02/03

In [ ]:
assignerPoints(papier='Sylvain Sainnier', # 
               numPoints=[123, 125, 141, 142, 157, 158, 159, 174, 175, 176, 192, 193]) # OK, papier 02/03

In [ ]:
assignerPoints(papier='Gilles Saulas', # 
               numPoints=[262, 263, 280, 281, 282, 283, 284, 299, 300, 301]) # OK, papier 02/03

In [ ]:
assignerPoints(naturalist='Cyrille Jallageas', # OK, naturalist 02/03 et 11/03
               numPoints=[194, 195, 196, 197, 198, # Smartphone
                          199, 200, 201, 202, 218, 219]) # Tablette

In [ ]:
assignerPoints(papier='Jean-François Carrias', # 
               numPoints=[23, 39, 40, 41, 42, 56, 57, 58, 59, 60]) # OK, papier 02/03

In [ ]:
assignerPoints(papier='Alex Clamens', # 
               numPoints=[55, 72, 73, 74, 75, 76, 90, 91]) # OK, papier 10/03

In [ ]:
assignerPoints(papier='Anne Citron', # 
               numPoints=[210, 211, 212, 228, 245, 246]) # OK, papier 11/03

In [ ]:
assignerPoints(papier='Camille Fasolin', # 
               numPoints=[88, 89, 105, 106, 122]) # OK, papier 11/03, tutorat Sylvain et Jean-François

In [ ]:
assignerPoints(naturalist='Matthieu Clément', # 
               numPoints=[157, 158, 159, 174, 175, 176, 192, 193]) # OK, 12/03

In [ ]:
assignerPoints(papier='Thibaut Brugerolle', # 
               numPoints=[148, 166, 182, 183, 184, 185]) # OK, papier (pb GPS revérifiés, naturalist=KO), 12/03

In [ ]:
assignerPoints(naturalist='Pierre Tourret', # 
               numPoints=[23, 39, 40, 41, 42, 56, 57, 58, 59, 60]) # ok, 12/03

In [ ]:
assignerPoints(papier='Sandra Robert', # 
               numPoints=[230, 231, 232, 249, 250]) # OK, 11/03 ; prévision de 5 points en plus 17/03

In [ ]:
assignerPoints(naturalist='Clément Rollant', # 
               numPoints=[113, 129, 130, 146, 147, 162]) # OK, naturalist

In [ ]:
assignerPoints(naturalist='Cyril Brunel', # 
               numPoints=[262, 263, 280, 281, 282, 283, 299, 300, 301]) # OK 18/03, naturalist 27/03

In [ ]:
assignerPoints(naturalist='Romain Riols', # 
               numPoints=[210, 211, 228, 245, 246, 247, 265, 266, 284]) # OK, naturalist, 01/04

In [ ]:
assignerPoints(papier='David Houston', # 
               numPoints=[]) # OK, papier /03

In [ ]:
assignerPoints(papier='Laurent Maly', # 
               numPoints=[]) # contacté, attente de réponse

In [ ]:
assignerPoints(papier='Patrick Mougel', # 
               numPoints=[]) # OK, papier ou naturalist ? 18/03

In [ ]:
assignerPoints(papier='Thyphaine Lyon', # 
               numPoints=[]) # OK, papier ou naturalist ? 26/03

In [ ]:
# Pour tests.
#assignerPoints(papier='Jean-Philippe', numPoints=[23, 39, 40])  # tests
#assignerPoints(naturalist='Jean-Philippe', numPoints=[145, 146, 147])  # tests
#assignerPoints(papier='François', numPoints=[109, 110, 112])  # tests
#assignerPoints(naturalist='François', numPoints=[41, 42])  # tests

In [ ]:
dfPoints.tail()

In [ ]:
# Bilan global
dfBilan = pd.DataFrame(data=[(len(dfPoints[dfPoints.papier == obseur]), len(dfPoints[dfPoints.naturalist == obseur])) \
                             for obseur in dfObseurs.nom] \
                            + [(len(dfPoints[dfPoints.papier.notnull()]), len(dfPoints[dfPoints.naturalist.notnull()])),
                               (len(dfPoints[dfPoints.papier.isnull()]), len(dfPoints[dfPoints.naturalist.isnull()]))],
                       columns=['Papier', 'Naturalist'],
                       index=list(dfObseurs.nom) + ['Assignés', 'Non assignés'])
dfBilan

In [ ]:
# En pourcentages ...
round(100 * dfBilan.loc[['Assignés', 'Non assignés']] / len(dfPoints), 1)

In [ ]:
dict(nbObseursAvecPoints=len(dfBilan[(dfBilan.Papier > 0) | (dfBilan.Naturalist > 0)].index)-2, nbObseursBrut=len(dfObseurs))

In [ ]:
# Uniquement les observateur présents le 31/03
dfBilan.loc[dfBilan.index.isin(dfObseurs[dfObseurs['31 mars'] != 'non'].nom)]

In [ ]:
df = dfObseurs[dfObseurs.nom.isin(dfBilan.loc[(dfBilan.Papier == 0) & (dfBilan.Naturalist == 0)].index)]
print('Observateurs n\'ayant pas encore choisi leurs points :', len(df))
print('* Les noms :', ', '.join(df.nom))
print('* Les emails :', ', '.join(df.eMail))

## 5) Export Excel

In [ ]:
attrTableFileName = 'ACDC/ACDC2019-AttributionsPoints.xlsx'

with pd.ExcelWriter(attrTableFileName) as xlWriter:
    dfPoints.reset_index().to_excel(xlWriter, sheet_name='AttribDétails', index=False)
    dfBilan.to_excel(xlWriter, sheet_name='AttribSynthèse')
    dfObseurs.to_excel(xlWriter, sheet_name='Observateurs', index=False)

HTML("""<p>Attributions : <a href='{fileName}' target="_blank">{fileName}</a></p>""" \
     .format(fileName=attrTableFileName))

## 6) Génération des KML individuels, avec cercles d'aide au relevé DS

In [ ]:
#Matrice de couleur des points selon l'assignation papier / naturalist du point
#                               Naturalist    
#                     papier    
KDefPointColors = { True  : { True : skml.Color.red,     False : skml.Color.lightgreen },
                    False : { True : skml.Color.fuchsia, False : skml.Color.cyan } }

def genererKml(dfPoints, dfZonePoly=None, observer=None, pointsColor=KDefPointColors,
               circles=[(25, 20), (100, 30), (200, 40)], circlesColor=skml.Color.white,
               title=None, postfix='cercles-stoceps', tgtDir='ACDC'):
    
    dfObsverPoints = dfPoints[(dfPoints.papier == observer) | (dfPoints.naturalist == observer)] if observer else dfPoints
    if len(dfObsverPoints) == 0:
        return None
    
    if not isinstance(pointsColor, dict):
        pointsColor = { papAss : { natAss : pointsColor for natAss in [True, False] } for papAss in [True, False] }
    
    kml = skml.Kml(name=title or 'Points ACDC 2019{}'.format(' '+observer if observer else ''))

    labelStyle = { papAss : { natAss : skml.LabelStyle(color=pointsColor[papAss][natAss], scale=1) \
                             for natAss in pointsColor[papAss] } for papAss in pointsColor }
    iconStyle = skml.IconStyle(icon=skml.Icon(href='http://maps.google.com/mapfiles/kml/shapes/placemark_circle.png'))
    ptStyle = { papAss : { natAss : skml.Style(labelstyle=labelStyle[papAss][natAss], iconstyle=iconStyle) \
                           for natAss in pointsColor[papAss] } for papAss in pointsColor }
    lineStyle = skml.LineStyle(color=skml.Color.red, width=3)

    if dfZonePoly is not None:
        ls = kml.newlinestring(name='Zone ACDC Cournols-Olloix JPD', extrude=1,
                               coords=dfZonePoly[['long', 'lat', 'alt']].values)
        ls.linestyle = lineStyle
        
    circleStyle = skml.LineStyle(color=circlesColor, width=1)

    obsver = ''
    for idx, sPt in dfObsverPoints.iterrows():
        
        # Point
        pt = kml.newpoint(name=str(idx), coords=[(sPt.longitude, sPt.latitude, 0)], extrude=1)
        pt.style = ptStyle[sPt.papier is not None][sPt.naturalist is not None]

        pt.description = 'Papier: {}, Naturalist: {} - UTM31 : lat={:.3f}km, long={:.3f}km' \
                         .format(sPt.papier or 'Personne', sPt.naturalist or 'Personne',
                                 sPt.longitude_utm, sPt.latitude_utm)
        
        # Cercles concentriques
        for r, n in circles:
            ls = kml.newlinestring(name='rayon={}m'.format(r), extrude=1,
                                   coords=kmlcircle.spoints(long=sPt.longitude, lat=sPt.latitude, meters=r, n=n, offset=0))
            ls.linestyle = circleStyle

    tgtKmlFileName = os.path.join(tgtDir, 'ACDC2019-{}{}-points{}.kml' \
                                          .format(observer.replace(' ', '').replace('-', '') +'-' if observer \
                                                  else '', len(dfObsverPoints), '-'+postfix if postfix else ''))
    kml.save(tgtKmlFileName)
    
    return tgtKmlFileName

In [ ]:
# Tous les observateurs, 1 par 1, cercles STOC EPS blancs.
html = "<table>"
for indObseur, obseur in enumerate(dfObseurs.nom):
    nomFicCarteAttrib = genererKml(dfPoints, dfZonePoly, obseur)
    html += """<tr><td>{num}</td>
                   <td style='text-align:left'>{obseur}</td>
                   <td  style='text-align:left'>{fileLink}</td></tr>""" \
            .format(num=indObseur+1, obseur=obseur,
                    fileLink="<a href='{fileName}' target='_blank'>{fileName}</a>".format(fileName=nomFicCarteAttrib) \
                             if nomFicCarteAttrib else '')
html += '</table>'

HTML(html)

In [ ]:
# Tous les observateurs, tous ensembles, cercles STOC EPS blancs.
nomFicCarteAttrib = genererKml(dfPoints, dfZonePoly)

HTML("""<p>Tous les points, cercles STOC-EPS : <a href='{fileName}' target="_blank">{fileName}</a></p>""" \
     .format(fileName=nomFicCarteAttrib))

In [ ]:
# Tous les observateurs, tous ensembles, cercles 300m éval. milieux fuschia.
nomFicCarteAttrib = genererKml(dfPoints, dfZonePoly, postfix='cercles-milieux', 
                               circles=[(300, 50)], circlesColor=skml.Color.fuchsia)

HTML("""<p>Tous les points, cercles milieux naturels : <a href='{fileName}' target="_blank">{fileName}</a></p>""" \
     .format(fileName=nomFicCarteAttrib))

In [ ]:
# Un seul observateur, cercles STOC EPS blancs.
#obseur = 'Jean-Philippe Meuret'
#nomFicCarteAttrib = genererKml(dfPoints, dfZonePoly, observer=obseur)
#
#HTML("""<p>Points de {obser}, cercles STOC-EPS : <a href='{fileName}' target="_blank">{fileName}</a></p>""" \
#     .format(obser=obseur, fileName=nomFicCarteAttrib))

In [ ]:
raise Exception('On s\'arrête ici !')

## 7) Cartographie des points obtenus

In [ ]:
#tiles, attr = 'http://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'osm'# OK
tiles, attr = 'https://{s}.tile.thunderforest.com/outdoors/{z}/{x}/{y}.png', 'thunderforest' # OK
#tiles, attr = 'http://{s}.tile.openstreetmap.fr/fradm/{z}/{x}/{y}.png', 'osm fr' # marche pô
#tiles, attr = 'https://{s}.tile.openstreetmap.fr/qa/{zoom}/{x}/{y}.png', 'osm fr' # marche pô
mp = folium.Map(tiles=tiles, attr=attr)

poly = folium.PolyLine(locations=[(lat, long) for long, lat in dfZonePoly[['long', 'lat']].itertuples(index=False)],
                       color='red', opacity=0.8, popup='Zone ACDC Cournols-Olloix JPD')
poly.add_to(mp)
for indPt, sPt in dfPoints.iterrows():
    mrk = folium.Marker(location=(sPt.latitude, sPt.longitude), 
                        popup=folium.Popup('Papier:{}, Naturalist:{} - UTM31: lat={:.3f}km, long={:.3f}km' \
                                           .format(sPt.papier or 'Personne', sPt.naturalist or 'Personne',
                                                   sPt.longitude_utm, sPt.latitude_utm)),
                        icon=folium.Icon(color='green', icon_color='black'))
    mrk.add_to(mp)
    
mp.fit_bounds(mp.get_bounds())
mp

# Génération du KML pour la matinée de formation du 31/03

In [ ]:
dfPointsForm = pd.DataFrame(data=[dict(papier='testeurs', naturalist=None, latitude=45.64776, longitude=3.04115,
                                       latitude_utm=0.0, longitude_utm=0.0, altitude=0, description='Point formation 1'),
                                  dict(papier='testeurs', naturalist=None, latitude=45.64752, longitude=3.04500,
                                       latitude_utm=0.0, longitude_utm=0.0, altitude=0, description='Point formation 2'),
                                  dict(papier='testeurs', naturalist=None, latitude=45.64661, longitude=3.04974,
                                       latitude_utm=0.0, longitude_utm=0.0, altitude=0, description='Point formation 3'),
                                  dict(papier='testeurs', naturalist=None, latitude=45.64908, longitude=3.04857,
                                       latitude_utm=0.0, longitude_utm=0.0, altitude=0, description='Point formation 4')],
                           index=range(1, 5))

In [ ]:
# Tous les points du test, cercles STOC EPS blancs.
nomFicCartePtsForm = genererKml(dfPointsForm, title='ACDC 2019 Points Formation 31/03', postfix='formation-cercles-stoceps')

HTML("""<p>Tous les points, cercles STOC-EPS : <a href='{fileName}' target="_blank">{fileName}</a></p>""" \
     .format(fileName=nomFicCartePtsForm))

# Génération de la grille initiale de points de relevés DS

1. charger KML donnant les limites de la zone
    * lecture KML => polygone zone en coordonnées sphériques
    * conversion / projection en coordonnées métriques : UTM 31
2. générer les points sur la grille et dans les limites de la zone, + certaine distance pour élargir ?
    * grille UTM 31 + qq 100m (qq entier)
    * 1er jet rectancle circonscrit à la zone (xMin, yMin, xMax, yMax) + marge
    * élimination des points hors du polygone de la zone ciblée + marge
3. exporter en KML

Cette grille devra être nettoyée des points inaccessibles, en forêt ...

## 1a) Charger KML donnant les limites de la zone

In [ ]:
kmlRoot = etree.ElementTree().parse('ACDC2019-limites-zone.kml')

In [ ]:
# On suppose que c'est le 1er polygône
plMark = kmlRoot.find('kml:Document/kml:Document/kml:Placemark', namespaces=kmlNameSpaces)
zonePoly = plMark.find('kml:Polygon/kml:outerBoundaryIs/kml:LinearRing/kml:coordinates',
                       namespaces=kmlNameSpaces).text.strip()
dfZonePoly = pd.DataFrame(data=[[float(v) for v in point.split(',')] for point in zonePoly.split(' ')],
                          columns=['long', 'lat', 'alt'])
dfZonePoly.head()

## 1b) Conversion coords polygône en métriques

In [ ]:
# Attention : Les colonnes sources (x,y) et (x_observateur,y_observateur)
#             sont bizarement des couples (lat, long), et pas l'inverse.
dfZonePoly[['xUtm', 'yUtm']] = \
  dfZonePoly[['long', 'lat']].apply(geoProjeter, srcProj=KProjWgs84, tgtProj=KProjUtm31, axis='columns')
dfZonePoly[['xUtm', 'yUtm']] = dfZonePoly[['xUtm', 'yUtm']].replace(KInfValues, np.nan) #, inplace=True)

In [ ]:
dfZonePoly.head()

In [ ]:
# Rectangle circonscrit à la zone
xUtmMin, yUtmMin, xUtmMax, yUtmMax = \
    dfZonePoly.xUtm.min(), dfZonePoly.yUtm.min(), dfZonePoly.xUtm.max(), dfZonePoly.yUtm.max()
xUtmMin, yUtmMin, xUtmMax, yUtmMax

## 2a) Détermination / choix de la taille des cellules de la grille

In [ ]:
# Le polygone de la zone (et sa surface en ha).
geoZonePoly = geometry.Polygon(shell=[(x, y) for x, y in dfZonePoly[['xUtm', 'yUtm']].itertuples(index=False)])

geoZonePoly.area / 10000

In [ ]:
# Nombre de points approximatif à répartir sur la zone.
nPoints = 150
txCouver = 60 # % ; rapide calcul après avoir dit : 100 points sur 2000 ha de milieux cibles (on vire les forêts)

In [ ]:
# Surface couverte par 1 point
surfPoint = geoZonePoly.area / nPoints
surfPoint / 10000, 'ha'

In [ ]:
# Soit un cercle de diamètre ...
deltaXYPoints = 2 * math.sqrt(surfPoint) * 100 / txCouver / math.pi
deltaXYPoints

In [ ]:
# Bon, on prend plutôt ...
deltaXYPoints = 500 #400

## 2b) Premier jet de points dans rectangle circonscrit + marge d'1 point

In [ ]:
# Rectangle circonscrit + marge d'1 point
xUtmMinR = xUtmMin - deltaXYPoints / 2
xUtmMaxR = xUtmMax + deltaXYPoints / 2
yUtmMinR = yUtmMin - deltaXYPoints / 2
yUtmMaxR = yUtmMax + deltaXYPoints / 2

In [ ]:
# Alignement sur une grille UTM à N m, avec ajustement par décalage en X, Y si besoin
uniteAlign = 100 # m
offsetX = 0
offsetY = 0

xUtmMinR = uniteAlign * math.floor(xUtmMinR / uniteAlign) + offsetX
yUtmMinR = uniteAlign * math.floor(yUtmMinR / uniteAlign) + offsetY
xUtmMaxR = uniteAlign * math.ceil(xUtmMaxR / uniteAlign) + offsetX
yUtmMaxR = uniteAlign * math.ceil(yUtmMaxR / uniteAlign) + offsetY

xUtmMinR, yUtmMinR, xUtmMaxR, yUtmMaxR

In [ ]:
dfPoints = pd.DataFrame(data=[dict(xUtm=x, yUtm=y) \
                              for y in np.arange(yUtmMaxR, yUtmMinR - deltaXYPoints, -deltaXYPoints) \
                              for x in np.arange(xUtmMinR, xUtmMaxR + deltaXYPoints, deltaXYPoints)])
dfPoints['numero'] = range(1, len(dfPoints)+1)
dfPoints[['long', 'lat']] = \
  dfPoints[['xUtm', 'yUtm']].apply(geoProjeter, srcProj=KProjUtm31, tgtProj=KProjWgs84, axis='columns')
dfPoints.set_index('numero', inplace=True)
dfPoints.head()

## 2c) Supprimer les points hors zone + marge

In [ ]:
# Marge en distance au delà de l'appartenance au polygône de la zone.
marginDist = deltaXYPoints * 1.0 #* 0.25

In [ ]:
def pointAroundZone(sXYPoint):
    point = geometry.Point(sXYPoint)
    #return geoZonePoly.contains(point) or point.distance(geoZonePoly) < marginDist
    return point.distance(geoZonePoly) < marginDist # Pas besoin de tester l'appartenance, distance() le fait.
dfPoints['aroundZoneExt'] = \
    dfPoints[['xUtm', 'yUtm']].apply(pointAroundZone, axis='columns')
len(dfPoints), len(dfPoints[dfPoints.aroundZoneExt])

In [ ]:
dfSelPoints = dfPoints[dfPoints.aroundZoneExt]
dfSelPoints.head()

## 3a) Cartographie des points obtenus

In [ ]:
tiles, attr = 'http://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'osm'# OK
#tiles, attr = 'https://{s}.tile.thunderforest.com/outdoors/{z}/{x}/{y}.png', 'thunderforest' # OK
#tiles, attr = 'http://{s}.tile.openstreetmap.fr/fradm/{z}/{x}/{y}.png', 'osm fr' # marche pô
#tiles, attr = 'https://{s}.tile.openstreetmap.fr/qa/{zoom}/{x}/{y}.png', 'osm fr' # marche pô
mp = folium.Map(tiles=tiles, attr=attr)

poly = folium.PolyLine(locations=[(lat, long) for long, lat in dfZonePoly[['long', 'lat']].itertuples(index=False)],
                       color='red', opacity=0.8, popup='Zone ACDC Cournols-Olloix JPD')
poly.add_to(mp)
for indPt, sPt in dfSelPoints.iterrows():
    mrk = folium.Marker(location=(sPt.lat, sPt.long), 
                        popup=folium.Popup('{} : lat={:.1f}, long={:.1f}'.format(indPt, sPt.lat, sPt.long)),
                        icon=folium.Icon(color='green', icon_color='black'))
    mrk.add_to(mp)
    
mp.fit_bounds(mp.get_bounds())
mp

## 3b) Export KML

In [ ]:
dfSelPoints.head()

In [ ]:
kml = skml.Kml(name='Points ACDC 2019 (dist={:.0f}m, marge={:.0f}m, n={})' \
               .format(deltaXYPoints, marginDist, len(dfSelPoints)))

In [ ]:
labelStyle = skml.LabelStyle(color=skml.Color.red, scale=1)
iconStyle = skml.IconStyle(icon=skml.Icon(href='http://maps.google.com/mapfiles/kml/shapes/placemark_circle.png'))
ptStyle = skml.Style(labelstyle=labelStyle, iconstyle=iconStyle)

lineStyle = skml.LineStyle(color=skml.Color.red, width=3)

In [ ]:
ls = kml.newlinestring(name='Zone ACDC Cournols-Olloix JPD', extrude=1,
                       coords=dfZonePoly[['long', 'lat', 'alt']].values)
                       #coords=[(long, lat, alt) for long, lat, alt in dfZonePoly[['long', 'lat', 'alt']].itertuples(index=False)])
ls.linestyle = lineStyle

for idx, sPt in dfSelPoints.iterrows():
    pt = kml.newpoint(name=str(idx), coords=[(sPt.long, sPt.lat, 0)], extrude=1)
    pt.style = ptStyle
    pt.description = 'lat={:.1f}, long={:.1f}, alt={:.0f}'.format(sPt.long, sPt.lat, 0)

In [ ]:
tgtKmlFileName = \
  'ACDC2019-{}points-et-limites-zone-d{:.0f}-m{:.0f}.kml'.format(len(dfSelPoints), deltaXYPoints, marginDist)
kml.save(tgtKmlFileName)

## 3c) Export Excel

In [ ]:
tgtXlsxFileName = \
  'ACDC2019-{}points-et-limites-zone-d{:.0f}-m{:.0f}.xlsx'.format(len(dfSelPoints), deltaXYPoints, marginDist)

dfSelPoints[['xUtm', 'yUtm', 'long', 'lat']].reset_index().to_excel(tgtXlsxFileName, index=False)